In [1]:
# Read the general DTS and save the taxonomy schema to PostgreSQL
from hibachiPostgreSQL import ReadAllGeneralDTSPathesInPostgre # For getting file general DTS pathes
from hibachixbrl import HibachiXmlElement, HibachiETtree
import pandas as pd
import sys
import os

In [3]:
# Get all the general DTS file pathes saved in PostgreSQL 
df_DTSPathes = ReadAllGeneralDTSPathesInPostgre.generalDTSPathes()

Connecting to the PostgreSQL database...
Connection is closed.


In [6]:
def readXSDinGeneralDTS(df_generalDTS):
    ''' Get all the TaxonomySchema, Linkbase, FootnoteLink, Roletype classes in .xsd file.'''

    for CurrentPathName in df_generalDTS['filepath']:
        #Get the extention of the current file
        CurrentFileName = os.path.split(CurrentPathName)[1]
        CurrentFileExtension = os.path.splitext(CurrentPathName)[1] 
        if CurrentFileExtension == '.xsd':
            
            # Dataframes for TaxonomySchema, Linkbase, RoleTypes
            df_TaxonomySchema = pd.DataFrame(columns=['filepath','TaxonomySchemaObj'])
            df_Linkbases      = pd.DataFrame(columns=['filepath','linkbaseObj'])
            df_RoleTypes      = pd.DataFrame(columns=['filepath','roletypeObj'])    
        
            try:
                root_ElementTree = HibachiETtree.etree_parse_remove_NS(CurrentPathName)
                if root_ElementTree != None:
                    
                    # List of tags for taxonomy schema, linkbase and roleType/Refs
                    schema_tags = ['import','include','schemaRef','linkbaseRef','roleRef', 'arcroleRef']
                    linkbase_tags = ['linkbase','schema']
                    search_tags = schema_tags + linkbase_tags
                    
                    # Search elements with the tags listed above and corresponding create instances
                    for empty_Var, search_tagName in enumerate(search_tags):
                        for schema_ElementTree in root_ElementTree.iter(search_tagName):
                            # Taxonomy Schema
                            if search_tagName in schema_tags:
                                #print(search_tagName)
                                df_TaxonomySchema = df_TaxonomySchema.append(\
                                    {'filepath':CurrentPathName,\
                                     'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.2 <linkbase>
                            elif search_tagName == 'linkbase':
                                df_Linkbases = df_Linkbases.append(\
                                    {'filepath':CurrentPathName,\
                                     'linkbaseObj':HibachiXmlElement.Linkbase(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.3 <roleType>, 5.1.4 <arcroleType>
#                             5.1.3 Defining custom role types - the <roleType> element
#                             The <roleType> element MUST be located among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @roleURI attribute identifies the @xlink:role attribute value that is being defined.
#                             5.1.4 Defining custom arc role types - the arcroleType element
#                             The <arcroleType> element MUST be among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @arcroleURI identifies the @xlink:arcrole attribute value that is being defined.

                            elif search_tagName == 'schema':
                                for annotation_ElementTree in schema_ElementTree.findall('annotation'):
                                    for appinfo_ElementTree in annotation_ElementTree.findall('appinfo'):
                                        
                                        roleType_Tags = ['roleType','arcroleType']
                                        for empty_Var, temp_tagName in enumerate(roleType_Tags):
                                            for temp_roleType_ElementTree in appinfo_ElementTree.findall(temp_tagName): #
                                                df_RoleTypes = df_RoleTypes.append(\
                                                    {'filepath':CurrentPathName,\
                                                     'roletypeObj':HibachiXmlElement.Roletype(CurrentFileName, temp_roleType_ElementTree)}\
                                                    ,ignore_index=True
                                                )
                                                
                                        roleRef_Tags = ['roleRef', 'arcroleRef']
                                        for empty_Var, temp_tagName in enumerate(roleRef_Tags):
                                            for temp_roleRef_ElementTree in appinfo_ElementTree.findall(temp_tagName): #
                                                df_TaxonomySchema = df_TaxonomySchema.append(\
                                                    {'filepath':CurrentPathName,\
                                                     'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
                                                    ,ignore_index=True
                                                )
                                        
                else:
                    print('No root element')

            except:
                print ('Error in reading:', CurrentPathName, sys.exc_info()[0], sys.exc_info()[1])
    return df_TaxonomySchema, df_Linkbases, df_RoleTypes

In [7]:
# Run the function
df_Taxonomy, df_Linkbase, df_RoleType = readXSDinGeneralDTS(df_generalDTS=df_DTSPathes)

Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\common\\2013-08-31/identificationAndOrdering_2013-08-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpcrp\\2013-08-31/jpcrp_cor_2013-08-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpcrp\\2013-08-31/jpcrp_rt_2013-08-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpcrp\\2015-03-31/jpcrp_cor_2015-03-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpcrp\\2015-03-31/jpcrp_rt_2015-03-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpcrp\\2015-03-31\\deprecated/jpcrp_d

Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpsps\\2015-03-31/jpsps_cor_2015-03-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpsps\\2015-03-31/jpsps_rt_2015-03-31.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpsps\\2017-02-28/jpsps_cor_2017-02-28.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpsps\\2017-02-28/jpsps_rt_2017-02-28.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpsps\\2018-02-28/jpsps_cor_2018-02-28.xsd'
No root element
Error when parsing <class 'FileNotFoundError'> [Errno 2] No such file or directory: 'EdinetGeneraltaxonomy\\jpsps\\2018-02-28/jpsps_rt_2018-02-28.xsd'
No root ele

In [117]:
# Unfold the Taxonomy Schema instace to Series and put it into dataframe
# Concate the unfolded dataframe to the original dataframe
#df_Taxonomy = pd.concat([df_Taxonomy.drop(['TaxonomySchemaObj'], axis=1), df_TaxonomySchema], axis=1)
df_Taxonomy = df_Taxonomy['TaxonomySchemaObj'].apply(lambda x: pd.Series(x.__dict__))
df_RoleType = df_RoleType['roletypeObj'].apply(lambda x: pd.Series(x.__dict__))

In [119]:
# Save the dataframe containing general DTS to PostgreSQL

# Connect to PostgreSQL
from sqlalchemy import create_engine
from config import config

try:
    params = config()
    engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**params))

    # Save the dataframe to PostgreSQL (make sure to type table names in lower cases)
    df_Taxonomy.to_sql('taxonomyschema_generaldts', con=engine, if_exists='replace', index=False)
    #df_RoleType.to_sql('roletype_generaldts', con=engine, if_exists='replace', index=False)
except:
    print ('Error:', sys.exc_info()[0], sys.exc_info()[1])

Error: <class 'sqlalchemy.exc.ProgrammingError'> (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO roletype_generaldts (tag, id, type, role, arcrole, base, originalpath, fragment_id, href_type, searchablepath, "roleURI", "arcroleURI", "cyclesAllowed", "childElementDict") VALUES (%(tag)s, %(id)s, %(type)s, %(role)s, %(arcrole)s, %(base)s, %(originalpath)s, %(fragment_id)s, %(href_type)s, %(searchablepath)s, %(roleURI)s, %(arcroleURI)s, %(cyclesAllowed)s, %(childElementDict)s)]
[parameters: ({'tag': 'roleType', 'id': 'rol_std_CabinetOfficeOrdinanceOnDisclosureOfCorporateInformationEtcFormNo2SecuritiesRegistrationStatement', 'type': None, 'role': None, 'arcrole': None, 'base': None, 'originalpath': None, 'fragment_id': None, 'href_type': 'No href', 'searchablepath': None, 'roleURI': 'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_std_CabinetOfficeOrdinanceOnDisclosureOfCorporateInformationEtcFormNo2SecuritiesRegistrationStatement', 'arcroleURI': None, 'cyclesAllowed